# Yapay Sinir Ağları


# Model

In [85]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import scale
from sklearn import model_selection
from sklearn.svm import SVR
from warnings import filterwarnings
filterwarnings('ignore')

In [86]:
hit=pd.read_csv("Hitters.csv")
df=hit.copy()
df=df.dropna()
dms=pd.get_dummies(df[["League","Division","NewLeague"]])
y=df["Salary"]
X_=df.drop(["Salary","League","Division","NewLeague"],axis=1).astype("float64")
X=pd.concat([X_,dms[["League_N","Division_W","NewLeague_N"]]],axis=1)
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [87]:
#dönüştürme işlemi yapalım
from sklearn.preprocessing import StandardScaler


scaler=StandardScaler()
scaler.fit(X_train)
X_train_scaled=scaler.transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [88]:
from sklearn.neural_network import MLPRegressor

In [89]:
mlp=MLPRegressor(hidden_layer_sizes=(100,20)).fit(X_train_scaled,y_train)
#hocam katman sayısını yukarıdakş sayıların sayısı belirle mesala burada 100,20 iki katman demek
#100,20,10 olsaydı 3 katman olurdu 
#buradakş sayılarda katmandaki eleman sayısını verir

In [90]:
mlp.hidden_layer_sizes
#gizeli katmannın size'inı verir

(100, 20)

In [91]:
mlp.n_layers_

4

# Tahmin

In [92]:
mlp.predict(X_train_scaled)[0:5]

array([ 69.17383494, 427.41062534, 305.83454753,  62.77604039,
        51.52606411])

In [93]:
#ilkel olarak test hatamizi hesaplayalım
np.sqrt(mean_squared_error(mlp.predict(X_test_scaled),y_test))

436.72122281213746

# Model Tuning

In [94]:
mlp_params={"alpha":[0.1,0.01,0.02,0.005],
            "hidden_layer_sizes":[(20,20),(100,50,150),(300,200,150)],
            "activation":["relu","logistic"]}

In [95]:
mlp_cv=GridSearchCV(mlp,mlp_params,cv=10)#.fit(X_train_scaled,y_train)

In [96]:
mlp_cv.fit(X_train_scaled,y_train)

GridSearchCV(cv=10, estimator=MLPRegressor(hidden_layer_sizes=(100, 20)),
             param_grid={'activation': ['relu', 'logistic'],
                         'alpha': [0.1, 0.01, 0.02, 0.005],
                         'hidden_layer_sizes': [(20, 20), (100, 50, 150),
                                                (300, 200, 150)]})

In [98]:
mlp_cv.best_params_

{'activation': 'relu', 'alpha': 0.005, 'hidden_layer_sizes': (100, 50, 150)}

In [100]:
mlp_tuned=MLPRegressor(alpha=0.005,hidden_layer_sizes=(100, 50, 150))#activatır defouly olarak relu olduğu için tekrardan yazmadık
mlp_tuned.fit(X_train,y_train)
np.sqrt(mean_squared_error(mlp_tuned.predict(X_test),y_test))

319.5935575519766